In [0]:
import os
import numpy as np
import glob
from PIL import Image
import zipfile
%tensorflow_version 2.x
import tensorflow as tf
# from shutil import copyfile

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/datsets/DRIVE.zip"\
    -O "/tmp/DRIVE.zip"

local_zip = '/tmp/DRIVE.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2020-03-03 11:26:41--  https://storage.googleapis.com/datsets/DRIVE.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29756930 (28M) [application/x-zip-compressed]
Saving to: ‘/tmp/DRIVE.zip’

/tmp/DRIVE.zip      100%[===================>]  28.38M   178MB/s    in 0.2s    

2020-03-03 11:26:41 (178 MB/s) - ‘/tmp/DRIVE.zip’ saved [29756930/29756930]



In [0]:
def GetAllChannels(imagewithchannels):
    redchannel=[]
    greenchannel=[]
    bluechannel=[]
    for i in range(len(imagewithchannels)):
        redchannel.append(imagewithchannels[i][:,:,0])
        greenchannel.append(imagewithchannels[i][:,:,1])
        bluechannel.append(imagewithchannels[i][:,:,2])
    return redchannel,greenchannel,bluechannel    

def create_patch(imgs,patch_size):
    a=0;
    ps=patch_size//2
    image_patches=[]
    for k in range(0,2):
        padded_imgs=np.pad(imgs[k],pad_width=ps,mode='edge')
        for i in range(0,imgs[k].shape[0]):
            for j in range(0,imgs[k].shape[1]):
                flatten_imgs=padded_imgs[i:i+patch_size,j:j+patch_size].ravel()
                imgs_mod_flatten=np.hstack((flatten_imgs))
                image_patches.append(imgs_mod_flatten)
    image_patches=np.vstack(image_patches)
    return image_patches

def getCombineChannels(red,green,blue):
    combinedPacthes=[]
    for i in range(red.shape[0]):
        combinedPacthes.append(np.array([red[i],green[i],blue[i]]))
    return(np.array(combinedPacthes))    

def reshapePatches(combinedpatches,reshapesize):
    newcombinedarray=[]
    end=combinedpatches.shape[0]
    for i in range(end):
        newcombinedarray.append(np.reshape(combinedpatches[i],reshapesize))
    return np.array(newcombinedarray)

Loading Training images

In [0]:
print("Loading Images.....")
images_training = [np.array(Image.open(file)) for file in glob.glob("/tmp/DRIVE/training/images/*.tif")]
print('Images Loaded.......')

Loading Images.....
Images Loaded.......


Loadinng Test imags

In [0]:
print("Loading Images.....")
images_testing = [np.array(Image.open(file)) for file in glob.glob("/tmp/DRIVE/test/images/*.tif")]
print('Images Loaded.......')

Loading Images.....
Images Loaded.......


Seperating Channels training

In [0]:
print('Seperating Channels.......')
(r_training, g_training,b_training) = GetAllChannels(images_training)
print('Channels seprated.......')

Seperating Channels.......
Channels seprated.......


Seperating Channels. testing

In [0]:
print('Seperating Channels.......')
(r_testing, g_testing, b_testing) = GetAllChannels(images_testing)
print('Channels seprated.......')

Seperating Channels.......
Channels seprated.......


Creating patches of red training

In [0]:
print('Creating patches for red channel')
patch_red_training = create_patch(r_training, 3)
print('Red channel patches created')

Creating patches for red channel
Red channel patches created


Creating patches of red testing

In [0]:
print('Creating patches for red channel')
patch_red_testing = create_patch(r_testing, 3)
print('Red channel patches created')

Creating patches for red channel
Red channel patches created


Creating patches of green training

In [0]:
print('Creating patches for green channel')
patch_green_training = create_patch(g_training, 3)
print('green channel patches created')

Creating patches for green channel
green channel patches created


Creating patches of green testing

In [0]:
print('Creating patches for green channel')
patch_green_testing = create_patch(g_testing, 3)
print('green channel patches created')

Creating patches for green channel
green channel patches created


Creating patches of blue training

In [12]:
print('Creating patches for blue channel')
patch_blue_training=create_patch(b_training,3)
print('Blue channel patches created')

Creating patches for blue channel
Blue channel patches created


Creating patches of blue testing

In [0]:
print('Creating patches for blue channel')
patch_blue_testing=create_patch(b_testing,3)
print('Blue channel patches created')

Creating patches for blue channel


Combinig all channel patches in single array training

In [0]:
print('Combinig all channel patches in single array')
combinedpatches_training=getCombineChannels(patch_red_training,patch_green_training,patch_blue_training)
print('All patches are combined and the shape is {}'.format(combinedpatches_training.shape))
# print(combinedpatches_training)

Combinig all channel patches in single array
All patches are combined and the shape is (659920, 3, 9)


Combinig all channel patches in single array testing

In [0]:
print('Combinig all channel patches in single array')
combinedpatches_testing=getCombineChannels(patch_red_testing,patch_green_testing,patch_blue_testing)
print('All patches are combined and the shape is {}'.format(combinedpatches_testing.shape))
# print(combinedpatches_testing)

Combinig all channel patches in single array
All patches are combined and the shape is (659920, 3, 9)


In [0]:
combinedpatches_testing[0].shape

(3, 9)

Reshaping to get required results training

In [0]:
print('Reshaping to get required results')
reshapedpatches_training=reshapePatches(combinedpatches_training,(3,3,3))
print('All patches are reshaped and the shape is {}'.format(reshapedpatches_training.shape))

Reshaping to get required results
All patches are reshaped and the shape is (659920, 3, 3, 3)


Reshaping to get required results testing

In [0]:
print('Reshaping to get required results')
reshapedpatches_testing=reshapePatches(combinedpatches_testing,(3,3,3))
print('All patches are reshaped and the shape is {}'.format(reshapedpatches_testing.shape))

Reshaping to get required results
All patches are reshaped and the shape is (659920, 3, 3, 3)


In [0]:
len(reshapedpatches_testing)

659920

In [0]:
print('Saving all patch data to file patchesknk.npy')
np.save('patchesknk_testing.npy',reshapedpatches_testing)
print('All patch data saved and can be access by variable \'reshapedpatches\'')

Saving all patch data to file patchesknk.npy
All patch data saved and can be access by variable 'reshapedpatches'


Manual image generate output training

In [0]:
print("Loading Images.....")
images_manual_training = [np.array(Image.open(file)) for file in glob.glob("/tmp/DRIVE/training/1st_manual/*.gif")]
print('Images Loaded.......')

Loading Images.....
Images Loaded.......


Manual image generate output testing

In [0]:
print("Loading Images.....")
images_manual_testing = [np.array(Image.open(file)) for file in glob.glob("/tmp/DRIVE/test/1st_manual/*.gif")]
print('Images Loaded.......')

Loading Images.....
Images Loaded.......


In [0]:
print(np.array(images_manual).shape)

(20, 584, 565)


In [0]:
output_training = []
output_training = np.array(images_manual_training).ravel()

In [0]:
output_testing = []
output_testing = np.array(images_manual_testing).ravel()

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (2, 2), activation='relu', input_shape=(3, 3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
x_train = x_train/ 255.0
# x_test = x_test/ 255.0
history = model.fit(images_manual_training, output_training, epochs = 4)

ValueError: ignored

In [0]:
a=np.reshape(combinedpatches[10013],(3,3,3))

In [0]:
a

In [0]:
reshapedpatches[10013]

In [0]:
reshapedpatches=reshapePatches(combinedpatches)

In [0]:
reshapedpatches[0]

In [0]:
a=[]
j=[[1,2,3],[1,2,3],[1,2,3]]
k=[[1,2,3],[1,2,3],[1,2,3]]
l=[[1,2,3],[1,2,3],[1,2,3]]
for i in range(3):
    a.append(np.array([j[i],k[i],l[i]]))

In [0]:
combinedpatches[0]

In [0]:
a=np.array(a)
a.shape

In [0]:
np.save('patches3n3.npy', data)

In [0]:
loaded_array = np.load('patches3n3.npy')

In [0]:
loaded_array.shape

In [0]:
a=[]
a.append(np.array([patch_red[0],patch_green[0],patch_blue[0]]))